Please enter the two cases below

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from numpy.linalg import inv
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import sklearn.metrics.pairwise as dist
from tqdm import tqdm
import seaborn as sns
%matplotlib inline
ratings=pd.read_csv('../../new_dataset/action_movies.csv',index_col=0)
movies=pd.read_csv('../../../../../dataset/ml-20m/movies.csv')
df_ref=pd.read_csv('../../new_dataset/correspondances_Id_movie')

In [4]:
#Définition du nombre de users et du nombre de films
user=0
movies_asked=10
user = input("Type a number between 0 and 1168 to indicate your IdUser : ")
movies_asked= input("Type the number of moovies you want to watch : ")
user=int(user)
movies_asked=int(movies_asked)


n_users = ratings.Id_Action.unique().shape[0]
n_movies = ratings.movie_Action.unique().shape[0]

filtre=list(ratings["movie_Action"][ratings["Id_Action"]==user].unique())

films_vus_par_u = ratings[ratings["movie_Action"].isin(filtre)]

#In a first we suppose assume that there is no user to remove but only movies to reindex
n_users=len(films_vus_par_u["Id_Action"].unique())
n_movies=len(films_vus_par_u["movie_Action"].unique())


jointure=pd.merge(movies,df_ref)
jointure.drop(['movieId', 'genres', 'imdbId', 'movieId_2', 'imdbId.1'],axis=1,inplace=True)
jointure=pd.merge(jointure,ratings)
jointure.drop(["movieId_ref","userId","Id_Action","rating"],axis=1,inplace=True)
jointure.head()
jointure.head()

movies_seen=jointure[jointure["movie_Action"].isin(filtre)]
print("Bravo, you have already seen ",str(n_movies) + " movies")
print("The list of movies I advice you is below")
print("this is the list of movies already seen : ",list(movies_seen["title"].unique()))

films_vus_par_u.drop(["userId","movieId_ref"],axis=1,inplace=True)

conversion_Id_movies_seen=pd.DataFrame({"movie_Action":list(films_vus_par_u["movie_Action"].unique()),"new_movies_Id":range(0,n_movies)})
result=pd.merge(films_vus_par_u, conversion_Id_movies_seen, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

matrice=np.zeros(( n_users,n_movies))
for line in result.itertuples():
    matrice[line[1], line[4]]=line[2]

user_similarity=dist.cosine_similarity(matrice)

a=list(ratings["movie_Action"].unique())
b=list(movies_seen["movie_Action"].unique())
movie_no_seen=[item for item in a if item not in b]
movie_no_seen=pd.DataFrame({"movie_Action" : movie_no_seen})
movie_no_seen=pd.merge(movie_no_seen,ratings,how='inner')

n_movies=len(movie_no_seen.movie_Action.unique())
#Il faut construire un nouveau index des Id_movies
conversion_Id_movies_no_seen=pd.DataFrame({"movie_Action":list(movie_no_seen["movie_Action"].unique()),"new_movies_Id":range(0,n_movies)})

result=pd.merge(movie_no_seen, conversion_Id_movies_no_seen, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

matrice_new=np.zeros(( 1168,n_movies))
for line in result.itertuples():
    matrice_new[line[3], line[6]]=line[5]
    
def pred_user(matrice_modele, user_similarity, k, user):    
    pred = np.zeros(matrice_modele.shape[1])
    top_k_users = np.argsort(user_similarity[:,user])[-1:-k-1:-1]  
    for i in tqdm(range(matrice_modele.shape[1])):
        pred[i]=user_similarity[user,:][top_k_users].dot(matrice_modele[:,i][top_k_users])
        pred[i]/=np.sum(np.abs(user_similarity[user,:][top_k_users]))+0.000001     
    return pred 

movies_adviced=np.argsort(pred_user(matrice_new, user_similarity, 15, user))[-1:-1-movies_asked:-1]
movies_adviced=list(result["movie_Action"][result["new_movies_Id"].isin(movies_adviced)].unique())
movies_adviced=list(ratings["movieId_ref"][ratings["movie_Action"].isin(movies_adviced)].unique())
movies_adviced=list(df_ref["movieId"][df_ref["movieId_ref"].isin(movies_adviced)].unique())

movies_adviced=list(movies["title"][movies["movieId"].isin(movies_adviced)].unique())
print("Here are movies you will love")
str(movies_adviced[::-1])[1:-1]

Type a number between 0 and 1168 to indicate your IdUser :  0
Type the number of moovies you want to watch :  10


Bravo, you have already seen  394 movies
The list of movies I advice you is below
this is the list of movies already seen :  ['Sudden Death (1995)', 'GoldenEye (1995)', 'Cutthroat Island (1995)', 'Money Train (1995)', 'Assassins (1995)', 'Mortal Kombat (1995)', 'Lawnmower Man 2: Beyond Cyberspace (1996)', 'From Dusk Till Dawn (1996)', 'Broken Arrow (1996)', 'Muppet Treasure Island (1996)', 'Rumble in the Bronx (Hont faan kui) (1995)', 'Bad Boys (1995)', 'Batman Forever (1995)', 'Congo (1995)', 'Crimson Tide (1995)', 'Desperado (1995)', 'Die Hard: With a Vengeance (1995)', 'First Knight (1995)', 'Johnny Mnemonic (1995)', 'Judge Dredd (1995)', 'Mighty Morphin Power Rangers: The Movie (1995)', 'Net, The (1995)', 'Species (1995)', 'Under Siege 2: Dark Territory (1995)', 'Waterworld (1995)', 'Drop Zone (1994)', 'Star Wars: Episode IV - A New Hope (1977)', 'Outbreak (1995)', 'Quick and the Dead, The (1995)', 'Specialist, The (1994)', 'Stargate (1994)', 'Tank Girl (1995)', 'Star Trek: Generat

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
100%|██████████| 667/667 [00:00<00:00, 29905.19it/s]

Here are movies you will love


"'Dark Knight, The (2008)', '300 (2007)', 'Casino Royale (2006)', 'V for Vendetta (2006)', 'King Kong (2005)', 'Serenity (2005)', 'Kung Fu Hustle (Gong fu) (2004)', 'Enter the Dragon (1973)', 'Equilibrium (2002)', 'Aliens (1986)'"

In [9]:
#Définition du nombre de users et du nombre de films
user=0
movies_asked=10
user = input("Type a number between 0 and 1168 to indicate your IdUser : ")
movies_asked= input("Type the number of moovies you want to watch : ")
user=int(user)
movies_asked=int(movies_asked)


n_users = ratings.Id_Action.unique().shape[0]
n_movies = ratings.movie_Action.unique().shape[0]

filtre=list(ratings["movie_Action"][ratings["Id_Action"]==user].unique())

films_vus_par_u = ratings[ratings["movie_Action"].isin(filtre)]

#In a first we suppose assume that there is no user to remove but only movies to reindex
n_users=len(films_vus_par_u["Id_Action"].unique())
n_movies=len(films_vus_par_u["movie_Action"].unique())


jointure=pd.merge(movies,df_ref)
jointure.drop(['movieId', 'genres', 'imdbId', 'movieId_2', 'imdbId.1'],axis=1,inplace=True)
jointure=pd.merge(jointure,ratings)
jointure.drop(["movieId_ref","userId","Id_Action","rating"],axis=1,inplace=True)
jointure.head()
jointure.head()

movies_seen=jointure[jointure["movie_Action"].isin(filtre)]
print("Bravo, you have already seen ",str(n_movies) + " movies")
print("The list of movies I advice you is below")
print("this is the list of movies already seen : ",list(movies_seen["title"].unique()))

films_vus_par_u.drop(["userId","movieId_ref"],axis=1,inplace=True)

conversion_Id_movies_seen=pd.DataFrame({"movie_Action":list(films_vus_par_u["movie_Action"].unique()),"new_movies_Id":range(0,n_movies)})
result=pd.merge(films_vus_par_u, conversion_Id_movies_seen, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

matrice=np.zeros(( n_users,n_movies))
for line in result.itertuples():
    matrice[line[1], line[4]]=line[2]

user_similarity=dist.cosine_similarity(matrice)

a=list(ratings["movie_Action"].unique())
b=list(movies_seen["movie_Action"].unique())
movie_no_seen=[item for item in a if item not in b]
movie_no_seen=pd.DataFrame({"movie_Action" : movie_no_seen})
movie_no_seen=pd.merge(movie_no_seen,ratings,how='inner')

n_movies=len(movie_no_seen.movie_Action.unique())
#Il faut construire un nouveau index des Id_movies
conversion_Id_movies_no_seen=pd.DataFrame({"movie_Action":list(movie_no_seen["movie_Action"].unique()),"new_movies_Id":range(0,n_movies)})

result=pd.merge(movie_no_seen, conversion_Id_movies_no_seen, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

matrice_new=np.zeros(( 1168,n_movies))
for line in result.itertuples():
    matrice_new[line[3], line[6]]=line[5]
    
def pred_user(matrice_modele, user_similarity, k, user):    
    pred = np.zeros(matrice_modele.shape[1])
    top_k_users = np.argsort(user_similarity[:,user])[-1:-k-1:-1]  
    for i in tqdm(range(matrice_modele.shape[1])):
        pred[i]=user_similarity[user,:][top_k_users].dot(matrice_modele[:,i][top_k_users])
        pred[i]/=np.sum(np.abs(user_similarity[user,:][top_k_users]))+0.000001     
    return pred 

movies_adviced=np.argsort(pred_user(matrice_new, user_similarity, 15, user))[-1:-1-movies_asked:-1]
movies_adviced=list(result["movie_Action"][result["new_movies_Id"].isin(movies_adviced)].unique())
movies_adviced=list(ratings["movieId_ref"][ratings["movie_Action"].isin(movies_adviced)].unique())
movies_adviced=list(df_ref["movieId"][df_ref["movieId_ref"].isin(movies_adviced)].unique())

movies_adviced=list(movies["title"][movies["movieId"].isin(movies_adviced)].unique())
print("Here are movies you will love")
str(movies_adviced[::-1])[1:-1]

Type a number between 0 and 1168 to indicate your IdUser :  2
Type the number of moovies you want to watch :  10


Bravo, you have already seen  648 movies
The list of movies I advice you is below
this is the list of movies already seen :  ['Sudden Death (1995)', 'GoldenEye (1995)', 'Cutthroat Island (1995)', 'Assassins (1995)', 'Mortal Kombat (1995)', 'From Dusk Till Dawn (1996)', 'Broken Arrow (1996)', 'Bad Boys (1995)', 'Batman Forever (1995)', 'Crimson Tide (1995)', 'Desperado (1995)', 'Die Hard: With a Vengeance (1995)', 'First Knight (1995)', 'Johnny Mnemonic (1995)', 'Judge Dredd (1995)', 'Prophecy, The (1995)', 'Species (1995)', 'Strange Days (1995)', 'Under Siege 2: Dark Territory (1995)', 'Waterworld (1995)', 'Star Wars: Episode IV - A New Hope (1977)', 'Quick and the Dead, The (1995)', 'Stargate (1994)', 'Tank Girl (1995)', 'Tales from the Crypt Presents: Demon Knight (1995)', 'Star Trek: Generations (1994)', 'Clear and Present Danger (1994)', 'Crow, The (1994)', 'Maverick (1994)', 'Speed (1994)', 'Timecop (1994)', 'True Lies (1994)', 'Faster Pussycat! Kill! Kill! (1965)', 'Highlander II

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 413/413 [00:00<00:00, 26467.54it/s]

Here are movies you will love


"'Scott Pilgrim vs. the World (2010)', 'Kingdom, The (2007)', 'Shooter (2007)', 'Kiss Kiss Bang Bang (2005)', 'Unleashed (Danny the Dog) (2005)', 'Kung Fu Hustle (Gong fu) (2004)', 'Incredibles, The (2004)', 'Enter the Dragon (1973)', 'Man on Fire (2004)', 'Grosse Pointe Blank (1997)'"